# 1. 生成器

## 推导式

### 列表推导式

### 字典推导式

### 生成器推导式

## 1.2 生成器表达式

## 生成器函数

# 迭代器

## 定义

1. 迭代器协议是指：对象必须提供一个next方法，执行方法要么返回迭代器中的下一项，要么就引起一个StopIteration异常，以终止迭代（只能往后走，不能往前退）
2. 可迭代对象：实现了迭代器协议的对象（实现方式:对象内部定义了一个__iter__()方法）
3. 协议是一种约定，可迭代对象实现了迭代器协议，Python的内部工具（如for循环，sum，min，max函数等）使用迭代器协议访问对象。

# 函数式编程

## 高阶函数
函数的参数为另一个函数名，或者函数的返回值为另一个函数（返回自己为递归），这种函数称为高阶函数

In [2]:
def func(a,b,f):
    return f(a)+f(b)
func(1,2,abs)

3

In [6]:
def foo():
    def inner():
        print('inner function')
    return inner
print(foo())
foo()()

<function foo.<locals>.inner at 0x000002544AB068B0>
inner function


## 内置高阶函数

- map
- filter
- reduce：是functools模块中的函数，需要导入
- sorted

## 闭包
在函数中可以（嵌套）定义另一个函数时，如果内部的函数引用了外部的函数的变量，则可能产生闭包。闭包可以用来在一个函数与一组“私有”变量之间创建关联关系。在给定函数被多次调用的过程中，这些私有变量能够保持其持久性。

### 闭包的定义

在函数中可以定义另一个函数时（函数嵌套），如果内部的函数引用了外部的函数的变量，则可能产生闭包。闭包可以用来在一个函数与一组“私有”变量之间创建关联关系。在给定函数被多次调用的过程中，这些私有变量能够保持其持久性。

### 闭包的条件

- 外部函数中定义了内部函数
- 外部函数是有返回值
- 返回值是：内部函数
- 内部函数还引用了外部函数的变量

闭包基本格式:
```python
def 外部函数():
    ...
    def 内部函数():
        ...
    return 内部函数
```

In [7]:
def func():
    a = 100
 
    def inner_func():
        b = 200
        print(a, b)
 
    return inner_func
 
 
x = func()
print(x)  # <function func.<locals>.inner_func at 0x0000021704CD9620>
x()  # 100 200
# 这样就可以直接调用inner_func函数，如果return不返回一个内部函数的话是不会有任何输出的

<function func.<locals>.inner_func at 0x000002544AB06790>
100 200


### 闭包的优缺点

优点：
- 可以同时使用内部函数变量和外部函数的变量
- 将外层函数的变量持久地保存在内存中。

缺点：
- 作用域没有那么直观
- 因为变量不会被垃圾回收所以有一定的内存占用问题。

## 装饰器
装饰器是在不改变原函数及调用方式的前提下添加新的功能。
特点：
1. 不改变原函数
2. 不改变原函数的调用

### 从需求开始
在不改变index函数源代码以及调用方式的前提下，为其添加统计运行时间的功能

In [12]:
import time
def index():
    time.sleep(3)
    print('index function')
index()

index function


### 方式一，修改源码

In [13]:
import time
def index():
    start = time.time()
    time.sleep(3)
    print('index function')
    stop= time.time()
    print(f'程序运行用时：{stop-start}')
index()

index function
程序运行用时：3.0062413215637207


此种方式，没有修改函数的调用方式，但是改了其源代码，所以此种方式不符合要求

### 方式二，在另一个函数种调用

In [11]:
import time

def index(x,y):
    time.sleep(3)
    print('index %s %s'%(x,y))
    
def wrapper():
    start=time.time()
    index(111,222)
    stop= time.time()
    print(f'程序运行用时：{stop-start}')

wrapper()

index 111 222
程序运行用时：3.0070884227752686


此种方式，没有修改函数的源代码，但是改了其调用方式，所以此种方式不符合要求

### 方式三，使用高阶函数

In [16]:
import time

def index():
    time.sleep(3)
    print('index function')
    
def wrapper(func):
    start=time.time()
    func()
    stop= time.time()
    print(f'程序运行用时：{stop-start}')

wrapper(index)

index function
程序运行用时：3.0142173767089844


此种方式，同样改变了函数的调用方式

### 方式四，高阶函数+闭包

In [18]:
import time

def index():
    time.sleep(3)
    print('index function')
    
def wrapper(func):
    def inner():
        start=time.time()
        func()
        stop= time.time()
        print(f'程序运行用时：{stop-start}')
    return inner

index = wrapper(index)
index()

index function
程序运行用时：3.0138185024261475


此种方式实现了我们的需求

### 使用@

在被装饰函数的上一行，使用@符号+装饰函数。此种方式也称为语法糖

In [24]:
import time

    
def wrapper(func):
    def inner():
        start=time.time()
        func()
        stop= time.time()
        print(f'程序运行用时：{stop-start}')
    return inner

@wrapper # 等效于index = wrapper(index)
def index():
    time.sleep(3)
    print('index function')
index()

index function
程序运行用时：3.0107123851776123


至此装饰器就完成了，所以装饰器=高阶函数+闭包+语法糖

### 无参装饰器

In [29]:
# 通用无参装饰函数
def wrapper(func):
    def inner():
        func()
    return inner


In [30]:
# 示例，计时器
import time
def timer(func):
    def inner():
        start = time.time()
        func()
        print(f'执行用时：{time.time()-start}秒')
    return inner

@timer
def foo():
    print('foo开始执行')
    time.sleep(3)
    print('foo执行完毕')

foo()

foo开始执行
foo执行完毕
执行用时：3.0075247287750244秒


### 有参数装饰器

In [49]:
# 示例，认证
import time
def auth(user,pwd,*args,**kwargs):
    def dector(func):
        def inner(*args,**kwargs):
            if user=='chen' and pwd=='123':
                func(*args,**kwargs)
            else:
                print('用户名或密码错误')
        return inner
    return dector

@auth('chen','123','abcd',a=10)
def index(request):
    print('欢迎登录你的首页')
    print(request)


index('request')

欢迎登录你的首页
request


### 装饰器中functools.wraps

In [51]:
def user_login_data(f):
    def wrapper(*args, **kwargs):
        return f(*args, **kwargs)
    return wrapper

# 用装饰器装饰以下两个函数

@user_login_data
def num1():
    print("aaa")
def num2():
    print("aaa")

print(num1.__name__) # 函数的名字被改变，因为本质上运行的就是wrapper函数
print(num2.__name__)

wrapper
num2


你写了一个装饰器作用在某个函数上，但是这个函数的重要的元信息比如名字、文档字符串、注解和参数签名都丢失了。

任何时候你定义装饰器的时候，都应该使用 functools 库中的 @wraps 装饰器来注解底层包装函数。

In [54]:
from functools import wraps

def user_login_data(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        return f(*args, **kwargs)
    return wrapper

# 用装饰器装饰以下两个函数

@user_login_data
def num1():
    print("aaa")
def num2():
    print("aaa")

print(num1.__name__) # 通过wraps装饰器就可以保留被装饰函数的元信息
print(num2.__name__)

num1
num2


- 在写装饰器时，请务必加上wraps装饰器

In [68]:
from functools import wraps

class Dector:
    def __init__(self,func):
        self.func = func
    
    def __call__(self, *args, **kwargs):
        self.func(*args,**kwargs)
@Dector
def index(a):
    print('index fun',a)

index(100)
print(index)

index fun 100


In [70]:
from functools import partial
def mun(a,b,c):
    sum = a+b+c
    return sum
s = mun(1,2,3)# 调用时需要传入三个参数
print(s)
mun_1 = partial(mun,b=2,c=3)# 包装一下原函数为一个新函数
s1 = mun_1(1)
s2 = mun_1(2)
print(s1)
print(s2)

6
6
7


In [75]:
from functools import partial
def sum(*args):
    # 这个函数只能计算n个数地和
    s = 0
    for n in args:
        s += n
    return s
s = sum(3,4,5,6)
print(s)

# 在原函数地基础上做一个扩展，让所有地值相加再加上10
func_1 = partial(sum,10)
s1 = func_1(3,4,5,6)
print(s1)

18
28


## 偏函数

在介绍函数参数的时候，我们讲到，通过设定参数的默认值，可以降低函数调用的难度。而偏函数也可以做到这一点。

### 偏函数的作用
- 如果一个函数有很多参数，并且会在很多地方调用，那么每次调用都要传入很多参数；是不是有一种办法固定住那些不会变化的参数
- 在不改变原函数的基础上，对函数功能进行扩展，类似于装饰器decorator的作用

# 面向对象编程

## 类与对象

## 访问限制

## 面向对象三大特性

### 继承

### 封装

### 多态

## 内置方法

## 元类

In [1]:
for i in {1,1,3}:
    print(i)

1
3
